In [342]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['gender_submission.csv', 'test.csv', 'train.csv']


In [343]:
dataset = pd.read_csv("../input/train.csv")

In [344]:
dataset.head()

PassengerId  Survived  Pclass    ...        Fare Cabin  Embarked
0            1         0       3    ...      7.2500   NaN         S
1            2         1       1    ...     71.2833   C85         C
2            3         1       3    ...      7.9250   NaN         S
3            4         1       1    ...     53.1000  C123         S
4            5         0       3    ...      8.0500   NaN         S

[5 rows x 12 columns]

In [345]:
dataset.groupby(["Sex", "Pclass"])["Age"].mean()

Sex     Pclass
female  1         34.611765
        2         28.722973
        3         21.750000
male    1         41.281386
        2         30.740707
        3         26.507589
Name: Age, dtype: float64

In [346]:
dataset.Age = dataset.groupby(['Pclass','Sex'])['Age'].transform(lambda x: x.fillna(x.mean()))

In [347]:
dataset.Age.isnull().any()

False

In [348]:
dataset["Cabin"]=dataset["Cabin"].apply(lambda x: "U" if x != x else x[0])

In [349]:
pd.get_dummies(dataset.loc[:, ~dataset.columns.isin(["Ticket", "Name"])]).groupby("Sex_female").mean()

PassengerId  Survived     ...      Embarked_Q  Embarked_S
Sex_female                            ...                            
0            454.147314  0.188908     ...        0.071057    0.764298
1            431.028662  0.742038     ...        0.114650    0.646497

[2 rows x 20 columns]

In [350]:
dataset[dataset.Embarked.isnull()]

PassengerId  Survived  Pclass    ...     Fare Cabin  Embarked
61            62         1       1    ...     80.0     B       NaN
829          830         1       1    ...     80.0     B       NaN

[2 rows x 12 columns]

In [351]:
dataset.loc[dataset.Embarked.isnull(), "Embarked"] = "U"

In [352]:
dataset.Name = dataset.Name.apply(lambda x: x.split(",")[1].split(".")[0])

In [353]:
dataset.groupby("Name").count()

PassengerId  Survived  Pclass    ...     Fare  Cabin  Embarked
Name                                            ...                          
 Capt                    1         1       1    ...        1      1         1
 Col                     2         2       2    ...        2      2         2
 Don                     1         1       1    ...        1      1         1
 Dr                      7         7       7    ...        7      7         7
 Jonkheer                1         1       1    ...        1      1         1
 Lady                    1         1       1    ...        1      1         1
 Major                   2         2       2    ...        2      2         2
 Master                 40        40      40    ...       40     40        40
 Miss                  182       182     182    ...      182    182       182
 Mlle                    2         2       2    ...        2      2         2
 Mme                     1         1       1    ...        1      1         1
 Mr                    517       517     517    ...      517    517       517
 Mrs                   125       125     125    ...      125    125       125
 Ms                      1         1       1    ...        1      1         1
 Rev                     6         6       6    ...        6      6         6
 Sir                     1         1       1    ...        1      1         1
 the Countess            1         1       1    ...        1      1         1

[17 rows x 11 columns]

In [354]:
for rare in ["Lady","Capt", "Col", "Don", "Dr", "Jonkheer", "Major", "Rev", "Sir", "the Countess"]:
    dataset.Name = dataset.Name.str.replace(rare, "Rare")

In [355]:
dataset.Name = dataset.Name.str.replace("Mlle", "Ms")
dataset.Name = dataset.Name.str.replace("Mme", "Mrs")


In [356]:
dataset.groupby("Name").mean()

PassengerId  Survived    ...         Parch       Fare
Name                              ...                         
 Master   414.975000  0.575000    ...      1.375000  34.703125
 Miss     408.884615  0.697802    ...      0.549451  43.797873
 Mr       454.499033  0.156673    ...      0.152805  24.441560
 Mrs      452.500000  0.793651    ...      0.825397  45.330290
 Ms       599.000000  1.000000    ...      0.000000  43.934733
 Rare     547.043478  0.347826    ...      0.086957  37.169748

[6 rows x 7 columns]

In [357]:
del dataset["PassengerId"]

In [358]:
dataset["IsAlone"] = dataset["SibSp"] + dataset["Parch"]
dataset["IsAlone"] = dataset["IsAlone"].apply(lambda x: 0 if x>0 else 1)

In [359]:
dataset.head()

Survived  Pclass   Name     Sex   ...       Fare  Cabin  Embarked IsAlone
0         0       3     Mr    male   ...     7.2500      U         S       0
1         1       1    Mrs  female   ...    71.2833      C         C       0
2         1       3   Miss  female   ...     7.9250      U         S       1
3         1       1    Mrs  female   ...    53.1000      C         S       0
4         0       3     Mr    male   ...     8.0500      U         S       1

[5 rows x 12 columns]

In [360]:
dataset.sum()

Survived                                                  342
Pclass                                                   2057
Name         Mr Mrs Miss Mrs Mr Mr Mr Master Mrs Mrs Miss ...
Sex         malefemalefemalefemalemalemalemalemalefemalefe...
Age                                                   26122.9
SibSp                                                     466
Parch                                                     340
Ticket      A/5 21171PC 17599STON/O2. 31012821138033734503...
Fare                                                  28693.9
Cabin       UCUCUUEUUUGCUUUUUUUUUDUAUUUCUUUBUUUUUUUUUUUUUU...
Embarked    SCSSSQSSSCSSSSSSQSSCSSQSSSCSQSCCQSCSCSSCSSCCQS...
IsAlone                                                   537
dtype: object

In [361]:
dataset["Ticket"][dataset["Ticket"].str[0].between("A", "Z")] = "Letter_Ticket"
dataset["Ticket"]="No_Letters"

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [362]:
dataset = pd.get_dummies(dataset, drop_first=True)

In [363]:
dataset

Survived  Pclass        Age     ...      Embarked_Q  Embarked_S  Embarked_U
0           0       3  22.000000     ...               0           1           0
1           1       1  38.000000     ...               0           0           0
2           1       3  26.000000     ...               0           1           0
3           1       1  35.000000     ...               0           1           0
4           0       3  35.000000     ...               0           1           0
5           0       3  26.507589     ...               1           0           0
6           0       1  54.000000     ...               0           1           0
7           0       3   2.000000     ...               0           1           0
8           1       3  27.000000     ...               0           1           0
9           1       2  14.000000     ...               0           0           0
10          1       3   4.000000     ...               0           1           0
11          1       1  58.000000     ...               0           1           0
12          0       3  20.000000     ...               0           1           0
13          0       3  39.000000     ...               0           1           0
14          0       3  14.000000     ...               0           1           0
15          1       2  55.000000     ...               0           1           0
16          0       3   2.000000     ...               1           0           0
17          1       2  30.740707     ...               0           1           0
18          0       3  31.000000     ...               0           1           0
19          1       3  21.750000     ...               0           0           0
20          0       2  35.000000     ...               0           1           0
21          1       2  34.000000     ...               0           1           0
22          1       3  15.000000     ...               1           0           0
23          1       1  28.000000     ...               0           1           0
24          0       3   8.000000     ...               0           1           0
25          1       3  38.000000     ...               0           1           0
26          0       3  26.507589     ...               0           0           0
27          0       1  19.000000     ...               0           1           0
28          1       3  21.750000     ...               1           0           0
29          0       3  26.507589     ...               0           1           0
..        ...     ...        ...     ...             ...         ...         ...
861         0       2  21.000000     ...               0           1           0
862         1       1  48.000000     ...               0           1           0
863         0       3  21.750000     ...               0           1           0
864         0       2  24.000000     ...               0           1           0
865         1       2  42.000000     ...               0           1           0
866         1       2  27.000000     ...               0           0           0
867         0       1  31.000000     ...               0           1           0
868         0       3  26.507589     ...               0           1           0
869         1       3   4.000000     ...               0           1           0
870         0       3  26.000000     ...               0           1           0
871         1       1  47.000000     ...               0           1           0
872         0       1  33.000000     ...               0           1           0
873         0       3  47.000000     ...               0           1           0
874         1       2  28.000000     ...               0           0           0
875         1       3  15.000000     ...               0           0           0
876         0       3  20.000000     ...               0           1           0
877         0       3  19.000000     ...               0           1           0
878         0       3  26.507589    

In [364]:
y = dataset.Survived
X = dataset.loc[:, ~dataset.columns.isin(["Survived", "SibSp", "Parch"])]

In [365]:
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
X = sc_x.fit_transform(X)

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [366]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    test_size=0.25,
                                                    random_state=0
                                                    )

In [367]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [368]:
classifier.score(X_test,y_test)

0.8026905829596412

In [369]:
classifier.score(X_train, y_train)

0.8248502994011976